In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ns8thivw
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ns8thivw
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=888b755fe668ecc83435d28f1566f3d3143ff62c574ac47f1a3c9e5c32e1ab4b
  Stored in directory: /tmp/pip-ephem-wheel-cache-hmy8itfe/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [3]:
%%cu
#include <stdio.h>

#define HANDLE_ERROR( err ) ( HandleError( err, __FILE__, __LINE__ ) )

static void HandleError( cudaError_t err, const char *file, int line )
{
    if (err != cudaSuccess)
      {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}



const long N = 100 ;

// CUDA Kernel for Vector Addition
__global__ void Vector_Addition ( const int *dev_a , const int *dev_b , int *dev_c)
{
      //Get the id of thread within a block
      unsigned short tid = threadIdx.x ;
     
      if ( tid < N ) // check the boundry condition for the threads
            dev_c [tid] = dev_a[tid] + dev_b[tid] ;

}


int main (void)
{
    
      int max = pow(2,29);
      clock_t start_t , end_t;
      int size = 0;
      double walltime, th;

      //Host array
      int Host_a[N], Host_b[N], Host_c[N];

      //Device array
      int *dev_a , *dev_b, *dev_c ;

      //Allocate the memory on the GPU
      HANDLE_ERROR ( cudaMalloc((void **)&dev_a , N*sizeof(int) ) );
      HANDLE_ERROR ( cudaMalloc((void **)&dev_b , N*sizeof(int) ) );
      HANDLE_ERROR ( cudaMalloc((void **)&dev_c , N*sizeof(int) ) );

      //fill the Host array with random elements on the CPU
 
      for(int i=0; i<N; i++)
      {
        Host_a[i]=rand()%1000;
      }
 
      for(int i=0; i<N; i++)
      {
        Host_b[i]=rand()%1000;
      }

      //Copy Host array to Device array
      HANDLE_ERROR (cudaMemcpy (dev_a , Host_a , N*sizeof(int) , cudaMemcpyHostToDevice));
      HANDLE_ERROR (cudaMemcpy (dev_b , Host_b , N*sizeof(int) , cudaMemcpyHostToDevice));

      //Make a call to GPU kernel
      Vector_Addition <<< 1, N  >>> (dev_a , dev_b , dev_c ) ;

      //Copy back to Host array from Device array
      HANDLE_ERROR (cudaMemcpy(Host_c , dev_c , N*sizeof(int) , cudaMemcpyDeviceToHost));

      //Display the result
      for ( int i = 0; i<N; i++ )
                  printf ("%d + %d = %d\n", Host_a[i] , Host_b[i] , Host_c[i] ) ;

      //Free the Device array memory
      cudaFree (dev_a) ;
      cudaFree (dev_b) ;
      cudaFree (dev_c) ;
 
      walltime =(end_t - start_t)/(double)CLOCKS_PER_SEC;
      th = (max*sizeof(int))/walltime;
      th = th/pow(10,9);

      printf("Time Taken: %lf\n",walltime);

      system("pause");
      return 0 ;

}

sh: 1: pause: not found
383 + 795 = 1178
886 + 570 = 1456
777 + 434 = 1211
915 + 378 = 1293
793 + 467 = 1260
335 + 601 = 936
386 + 97 = 483
492 + 902 = 1394
649 + 317 = 966
421 + 492 = 913
362 + 652 = 1014
27 + 756 = 783
690 + 301 = 991
59 + 280 = 339
763 + 286 = 1049
926 + 441 = 1367
540 + 865 = 1405
426 + 689 = 1115
172 + 444 = 616
736 + 619 = 1355
211 + 440 = 651
368 + 729 = 1097
567 + 31 = 598
429 + 117 = 546
782 + 97 = 879
530 + 771 = 1301
862 + 481 = 1343
123 + 675 = 798
67 + 709 = 776
135 + 927 = 1062
929 + 567 = 1496
802 + 856 = 1658
22 + 497 = 519
58 + 353 = 411
69 + 586 = 655
167 + 965 = 1132
393 + 306 = 699
456 + 683 = 1139
11 + 219 = 230
42 + 624 = 666
229 + 528 = 757
373 + 871 = 1244
421 + 732 = 1153
919 + 829 = 1748
784 + 503 = 1287
537 + 19 = 556
198 + 270 = 468
324 + 368 = 692
315 + 708 = 1023
370 + 715 = 1085
413 + 340 = 753
526 + 149 = 675
91 + 796 = 887
980 + 723 = 1703
956 + 618 = 1574
873 + 245 = 1118
862 + 846 = 1708
170 + 451 = 621
996 + 921 = 1917
281 + 555 = 83

**Matrix Multiplication**

In [4]:
%%cu
#include<stdio.h>
#include<cuda.h>
#define row1 4 /* Number of rows of first matrix */
#define col1 4 /* Number of columns of first matrix */
#define row2 4 /* Number of rows of second matrix */
#define col2 4 /* Number of columns of second matrix */

__global__ void matproduct(int *l,int *m, int *n)
{
    int x=blockIdx.x;
    int y=blockIdx.y;
    int k;
  
n[col2*y+x]=0;
for(k=0;k<col1;k++)
   {
    n[col2*y+x]=n[col2*y+x]+l[col1*y+k]*m[col2*k+x];
   }
}

int main()
{
    int a[row1][col1] = { { 4, 2, 7, 1 },
                       { 3, 8, 2, 3 },
                       { 6, 9, 12, 4 },
                       { 10, 7, 2, 4 } };
    int b[row2][col2] = { { 3, 5, 4, 9 },
                       { 7, 1, 6, 13 },
                       { 11, 5, 9, 2 },
                       { 4, 7, 12, 3 } };
    int c[row1][col2];
    int *d,*e,*f;
    int i,j;

    /*
    printf("\n Enter elements of first matrix of size 2*3\n");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col1;j++)
            {
                scanf("%d",&a[i][j]);
            }
    }
    printf("\n Enter elements of second matrix of size 3*2\n");
        for(i=0;i<row2;i++)
        {
            for(j=0;j<col2;j++)
                {
                    scanf("%d",&b[i][j]);
                }
        }
    */

    cudaMalloc((void **)&d,row1*col1*sizeof(int));
    cudaMalloc((void **)&e,row2*col2*sizeof(int));
    cudaMalloc((void **)&f,row1*col2*sizeof(int));

 cudaMemcpy(d,a,row1*col1*sizeof(int),cudaMemcpyHostToDevice);
 cudaMemcpy(e,b,row2*col2*sizeof(int),cudaMemcpyHostToDevice);

dim3 grid(col2,row1);
/* Here we are defining two dimensional Grid(collection of blocks) structure. Syntax is dim3 grid(no. of columns,no. of rows) */

    matproduct<<<grid,1>>>(d,e,f);

 cudaMemcpy(c,f,row1*col2*sizeof(int),cudaMemcpyDeviceToHost);
    printf("\nProduct of two matrices:\n ");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col2;j++)
        {
              printf("%d\t",c[i][j]);
        }
        printf("\n");
    }

    cudaFree(d);
    cudaFree(e);
    cudaFree(f);

    return 0;
}


Product of two matrices:
 107	64	103	79	
99	54	114	144	
229	127	234	207	
117	95	148	197	

